<a href="https://colab.research.google.com/github/Clara-Fakhry/content/blob/main/Project490backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4


In [ ]:
import openai
import os

# Set your API key directly
openai.api_key = "Your api key"
# Or, if you want to avoid hard-coding the API key:
# from getpass import getpass
# openai.api_key = getpass("Enter your OpenAI API Key: ")


In [ ]:
def get_chatgpt_response(prompt, model="gpt-4o"):
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    # Extract and return the response text
    return response.choices[0].message['content']


In [ ]:
!pip install PyMuPDF openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 93.1 MB/s eta 0:00:00


In [ ]:
import fitz

def extract_text_from_pdf(pdf_path):
    # Initialize an empty string to store text
    text = ""
    # Open the PDF file
    with fitz.open(pdf_path) as doc:
        # Loop through each page and extract text
        for page in doc:
            text += page.get_text()
    return text

def get_cv_features(cv_text):
    # Define a prompt to guide GPT-4 in extracting specific CV features
    prompt = f"""
    Extract the following details from the CV text below and structure them in a clear format:

    1. Contact Information (e.g., name, phone number, email, address).
    2. Summary or Objective (if available).
    3. Education (degrees, institutions, graduation dates, GPA or academic achievements).
    4. Work Experience (company names, job titles, responsibilities, achievements, dates).
    5. Skills (both technical and soft skills).
    6. Certifications or Training (certificate names, issuing organizations, dates).
    7. Achievements and Awards (e.g., recognitions, honors, major accomplishments).
    8. Extracurricular Activities (e.g., clubs, volunteer work, sports).
    9. Languages (languages spoken and proficiency levels).
    10. Projects (if any relevant projects are mentioned).
    11. References (if any references are included).


    CV Text:
    {cv_text}
    """

    # Use OpenAI's API to process the prompt


    # Extract the content of the response
    extracted_info = get_chatgpt_response(prompt)
    return extracted_info

# Path to the CV PDF file
pdf_path = 'Charbel-CV.pdf'

# Extract text from the CV PDF
cv_text = extract_text_from_pdf(pdf_path)

# Get structured information
cv_features = get_cv_features(cv_text)

#print("Extracted CV Information:\n", cv_features)

In [ ]:
!pip install PyMuPDF openai tqdm

In [ ]:


# Function to generate potential job roles based on CV features
def generate_potential_roles(cv_features):
    prompt = f"""
    Given the following CV features, suggest a list of potential job roles that may suit the candidate.
    Consider roles that align with the candidate's skills, experience, and education, and provide a brief
    explanation of why each role may be a good fit.

    CV Features:
    {cv_features}
    """


    potential_roles = get_chatgpt_response(prompt)
    return potential_roles

# Function to analyze strengths and gaps for each role
def analyze_roles_with_cv(cv_features, potential_roles):
    prompt = f"""
    Based on the following potential job roles and CV features, identify the candidate's strengths and gaps for each role.
    For each role, outline which skills or experiences are strong matches and which may require development. Minimize your wordings as much as possible while keeping the information concise.

    Potential Job Roles:
    {potential_roles}

    CV Features:
    {cv_features}

    Potential Job Roles:
    {potential_roles}
    """


    role_analysis = get_chatgpt_response(prompt)
    return role_analysis

# Path to the CV PDF file
pdf_path = 'Charbel-CV.pdf'

# Extract text from the CV PDF
cv_text = extract_text_from_pdf(pdf_path)

# Step 1: Extract structured information from the CV
cv_features = get_cv_features(cv_text)
#print("Extracted CV Information:\n", cv_features)

# Step 2: Generate potential job roles based on CV features
potential_roles = generate_potential_roles(cv_features)
#print("\nPotential Job Roles:\n", potential_roles)

# Step 3: Analyze strengths and gaps for each potential job role
role_analysis = analyze_roles_with_cv(cv_features, potential_roles)
#print("\nRole Analysis (Strengths and Gaps):\n", role_analysis)


In [ ]:
def get_career_path_descriptions(potential_roles):
    prompt = f"""
    For each of the following job roles, provide a detailed description including:
    - Key skill requirements
    - Current industry trends
    - Potential growth areas
    Minimize your wordings as much as possible while keeping the information concise.

    Job Roles:
    {potential_roles}
    """


    career_path_descriptions = get_chatgpt_response(prompt)
    return career_path_descriptions

def calculate_skill_match(cv_features, refined_recommendations):
    prompt = f"""
    For each of the refined job roles provided, calculate a skill match percentage based on the candidate's skills from their CV.
    Match each skill mentioned in the CV to the skills required for the job role, and return a percentage match score.
    Additionally, provide brief reasoning for the score based on matching and missing skills, but when you give me the refined recommendations and the refined skill match percentages, dont give me the skill match percentages of the old recommendations
    Minimize your wordings as much as possible while keeping the information concise.

    CV Skills:
    {cv_features}

    Refined Job Roles:
    {refined_recommendations}
    """

    skill_match_data = get_chatgpt_response(prompt)
    return skill_match_data



In [ ]:



# Step 2: Collect user feedback and refine recommendations with skill match percentages
def collect_user_feedback_and_refine_recommendations(recommendations, potential_roles, role_analysis, cv_features):
    # Get user rating and feedback
    try:
        rating = int(input("On a scale of 1 to 10, how would you rate the relevance of these career suggestions? "))
        if rating < 1 or rating > 10:
            print("Please enter a rating between 1 and 10.")
            return collect_user_feedback_and_refine_recommendations(recommendations, potential_roles, role_analysis, cv_features)
    except ValueError:
        print("Please enter a valid integer between 1 and 10.")
        return collect_user_feedback_and_refine_recommendations(recommendations, potential_roles, role_analysis, cv_features)

    feedback = input("Provide any comments on the career suggestions (mention likes/dislikes): ")

    # Refine recommendations based on user feedback and add skill match percentages
    prompt = f"""
    Based on the initial recommendations and the user's feedback, provide refined career suggestions.
    Adjust recommendations based on the candidate's preferences and skill match percentage.
    Minimize your wordings as much as possible while keeping the information concise.

    Initial Recommendations:
    {recommendations}

    User Feedback:
    Rating: {rating}/10
    Comments: {feedback}

    CV Features:
    {cv_features}

    Previous Potential Job Roles:
    {potential_roles}

    Previous Role Analysis:
    {role_analysis}
    """

    # Generate refined recommendations using the feedback

    refined_recommendations = get_chatgpt_response(prompt)

    # Calculate skill match percentages for refined recommendations
    refined_skill_match_data = calculate_skill_match(cv_features, potential_roles)

    return {"rating": rating, "feedback": feedback, "refined_recommendations": refined_recommendations, "refined_skill_match_data": refined_skill_match_data}

# Step 3: Main function for recommendation engine with initial and refined skill match percentages
def career_recommendation_engine_with_feedback_and_skill_match(pdf_path):
    # Extract and analyze CV information as in previous steps
    cv_text = extract_text_from_pdf(pdf_path)
    cv_features = get_cv_features(cv_text)
    potential_roles = generate_potential_roles(cv_features)
    role_analysis = analyze_roles_with_cv(cv_features, potential_roles)
    career_path_descriptions = get_career_path_descriptions(potential_roles)

    # Calculate initial skill match percentages
    initial_skill_match_data = calculate_skill_match(cv_features, potential_roles)

    # Display initial recommendations with skill match percentages
    initial_recommendations = f"""
    Career Recommendations for the Candidate:

    Extracted CV Features:
    {cv_features}

    Recommended Job Roles:
    {potential_roles}

    Role Analysis (Strengths and Gaps):
    {role_analysis}

    Career Path Descriptions (with skill requirements, trends, and growth areas):
    {career_path_descriptions}

    Initial Skill Match Percentages:
    {initial_skill_match_data}
    """
    print(initial_recommendations)

    # Collect feedback and refine recommendations with skill match
    user_feedback = collect_user_feedback_and_refine_recommendations(
        recommendations=initial_recommendations,
        potential_roles=potential_roles,
        role_analysis=role_analysis,
        cv_features=cv_features
    )

    # Display refined recommendations and skill match percentages
    print("\nRefined Career Recommendations:\n", user_feedback["refined_recommendations"])
    #print("\nRefined Skill Match Percentages:\n", user_feedback["refined_skill_match_data"])

    return user_feedback["refined_recommendations"], user_feedback["refined_skill_match_data"]

# Path to the CV PDF file
pdf_path = 'Charbel-CV.pdf'

# Run the recommendation engine with skill match calculation and feedback refinement
#refined_recommendations, refined_skill_match = career_recommendation_engine_with_feedback_and_skill_match(pdf_path)


In [ ]:


# Function to identify skill gaps and recommend resources based on refined career suggestions
def identify_skill_gaps_and_recommend_resources(cv_features, refined_recommendations):
    # Define prompt to analyze gaps and recommend resources
    prompt = f"""
    Based on the user's current skill set in their CV and the refined career recommendations, identify any skill gaps for each recommended career path.
    For each skill gap, provide actionable recommendations including courses, certifications, project ideas, and other resources to help the user develop these skills.

    The recommendations should include:
    - Specific learning platforms (like Coursera, LinkedIn Learning, Udemy, edX)
    - Suggested project ideas to apply the skills practically
    - Mentorship or networking suggestions if applicable
    Minimize your wordings as much as possible while keeping the information concise.

    User's Current Skills:
    {cv_features}

    Refined Career Recommendations and Required Skills:
    {refined_recommendations}
    """

    # Call the OpenAI API to get skill gap analysis and recommendations


    # Extract the content of the response for skill gap recommendations
    skill_gap_recommendations = get_chatgpt_response(prompt)

    return skill_gap_recommendations

# Main function to execute skill gap analysis and resource recommendation
def skill_gap_analysis_with_recommendations(pdf_path):
    # Extract and analyze CV information
    cv_text = extract_text_from_pdf(pdf_path)
    cv_features = get_cv_features(cv_text)

    # Generate initial recommendations and refine based on feedback
    refined_recommendations, refined_skill_match_data = career_recommendation_engine_with_feedback_and_skill_match(pdf_path)


    # Perform skill gap analysis on refined recommendations
    skill_gap_recommendations = identify_skill_gaps_and_recommend_resources(cv_features, refined_recommendations)

    # Display the skill gap recommendations
    print("\nSkill Gap Analysis and Recommendations:\n")
    print(skill_gap_recommendations)

# Run the skill gap analysis with resource recommendations
pdf_path = 'Charbel-CV.pdf'
#skill_gap_analysis_with_recommendations(pdf_path)


In [ ]:
# Function to get a motivational quote
def get_motivational_quote():
    quote_prompt = "Give me an uplifting and motivational quote to inspire someone who's working on their career goals."
    quote= get_chatgpt_response(quote_prompt)
    return quote

In [ ]:
# Global dictionary to store completed steps for each goal
goal_completion_tracker = {}

# Function to calculate progress for a specific goal
def calculate_goal_progress(goal_steps, completed_steps):
    total_steps = len(goal_steps)
    completed_count = len(completed_steps)
    progress_percentage = (completed_count / total_steps) * 100
    return progress_percentage

# Main function for career goal tracking
def career_goal_tracking(pdf_path, career_titles=None, smart_goals_dict=None):
    global goal_completion_tracker  # Use the global tracker to retain the completed steps across interactions

    # If no career_titles and smart_goals_dict are provided, generate them
    if career_titles is None or smart_goals_dict is None:
        # Step 1: Extract CV information and generate refined recommendations
        cv_text = extract_text_from_pdf(pdf_path)
        cv_features = get_cv_features(cv_text)
        refined_recommendations, _ = career_recommendation_engine_with_feedback_and_skill_match(pdf_path)

        # Extract career titles
        career_titles = extract_career_titles(refined_recommendations)

        # Step 2: Generate SMART goals and steps for all career recommendations
        smart_goals_dict = {}
        print("\n--- Generating SMART Goals for Career Recommendations ---")
        for title in career_titles:
            print(f"Generating goals for: {title}")
            smart_goals = generate_smart_goals(title)
            smart_goals_dict[title] = smart_goals
            print(f"\nSMART Goals for {title}:")
            print(smart_goals)
            print("--------------------------------------")
        print("\nSMART goals and steps are ready!")

    # Step 3: Start user interaction
    # Display career titles as a clean list
    print("\n--- Refined Career Recommendations ---")
    for i, title in enumerate(career_titles, 1):
        print(f"{i}. {title}")
    print("--------------------------------------")

    # User selects a career recommendation
    selected_index = int(input("\nSelect a career recommendation (enter the number): ")) - 1
    selected_recommendation = career_titles[selected_index]
    print(f"\nSelected Career Recommendation: {selected_recommendation}")

    # Display SMART goals for the selected recommendation
    print("\n--- SMART Goals for Selected Recommendation ---")
    smart_goals_text = smart_goals_dict[selected_recommendation]

    # Split SMART goals into individual goals using a consistent delimiter
    goals = smart_goals_text.split("\n### Goal")  # Split by "### Goal" to isolate each goal

    # Prepend "### Goal" to each split goal for proper formatting
    goals = [f"### Goal{goal}" for goal in goals if goal.strip()]  # Ensure formatting remains intact

    # Check if goals are properly extracted
    if not goals or all(goal.strip() == "" for goal in goals):
        print("No SMART goals were generated for this recommendation. Please try again.")
        return  # Exit gracefully if no goals are found

    # Display all extracted goals with proper enumeration
    for i, goal in enumerate(goals, 1):
        # Show only the title of each goal
        title = goal.strip().splitlines()[0] if goal.strip() else "Untitled Goal"
        print(f"{i}. {title}")
    print("-----------------------------------------------")

    # User selects a goal with validation
    while True:
        try:
            goal_index = int(input("\nSelect a goal (enter the number): ")) - 1
            if goal_index < 0 or goal_index >= len(goals):
                raise ValueError("Invalid selection. Please enter a valid number.")
            break
        except ValueError as e:
            print(f"Invalid input: {e}. Please try again.")

    # Isolate the selected goal
    selected_goal = goals[goal_index].strip()

    # Display the selected goal
    print(f"\nSelected Goal:\n{selected_goal}")

    # Extract steps for the selected goal
    steps = []  # Initialize steps as an empty list

    # Attempt to locate a steps section by searching for common headers
    possible_headers = ["**Steps to Achieve:**", "**Steps to Achieve the Goal:**", "**Steps:**", "**Steps to Achieve this Goal:**"]
    steps_start = -1  # Initialize with -1 to indicate no header found

    # Find the starting position of the steps section
    for header in possible_headers:
        steps_start = selected_goal.find(header)
        if steps_start != -1:  # If a header is found, stop the search
            break

    # If a steps section header is found, extract and parse the steps
    if steps_start != -1:
        # Extract the portion of the text after the header
        steps_section = selected_goal[steps_start + len(header):].strip()

        # Parse steps: Look for numbered lines and stop at unrelated sections
        for line in steps_section.split("\n"):
            line = line.strip()
            if line and line[0].isdigit():  # Identify numbered steps
                steps.append(line)
            elif line.startswith("**"):  # Stop parsing at a new section header
                break
    else:
        # Handle case where no steps header is found
        print("\nNo steps section header detected in the goal. Please verify the goal format.")

    # Display steps for the selected goal
    if not steps:
        print("\nNo steps available for this goal. Please check the goal setup.")
    else:
        print("\n--- Steps for the Selected Goal ---")
        for i, step in enumerate(steps, 1):  # Display steps with proper numbering
            print(f"- {step}")
        print("-----------------------------------")

        # Retrieve previously completed steps from the tracker
        if selected_goal not in goal_completion_tracker:
            goal_completion_tracker[selected_goal] = []

        completed_steps = goal_completion_tracker[selected_goal]

        # User marks completed steps with validation
        while True:
            try:
                # Prompt user for completed steps
                completed_input = input("\nEnter the numbers of completed steps (comma-separated): ").strip()

                # Ensure input is not empty
                if not completed_input:
                    raise ValueError("You must enter at least one step number.")

                # Parse input and convert to zero-based indices
                completed_indices = [int(idx.strip()) - 1 for idx in completed_input.split(",")]

                # Validate indices against the steps list
                if any(idx < 0 or idx >= len(steps) for idx in completed_indices):
                    raise ValueError("Some indices are out of range. Please ensure your input matches the listed steps.")

                # Check for duplicate steps (i.e., the user should not re-enter steps they've already completed)
                already_completed = [steps[idx] for idx in completed_indices if steps[idx] in completed_steps]
                if already_completed:
                    raise ValueError(f"You've already marked these steps as completed: {', '.join(already_completed)}. Please only enter new steps.")

                # If all validations pass, break the loop
                break
            except ValueError as e:
                print(f"Invalid input: {e}. Please try again.")

        # Retrieve completed steps and update the tracker
        completed_steps.extend([steps[idx] for idx in completed_indices])

        # Store the updated list in the global tracker
        goal_completion_tracker[selected_goal] = completed_steps

        # Display completed steps
        print("\nCompleted Steps:")
        for step in completed_steps:
            print(f"- {step}")

    # Step 5: Calculate progress
    progress = calculate_goal_progress(steps, completed_steps)
    print(f"\nGoal Completion Progress: {progress:.2f}%")

    # Provide motivational advice
    if progress < 100:
        incomplete_steps = [step for step in steps if step not in completed_steps]
        print("\nMotivational Feedback: Keep pushing! You’re making progress.")
        print("Here are additional resources to help you complete the remaining steps:")
        resource_prompt = f"""
        Provide motivational advice and resources for the following incomplete steps:
        {incomplete_steps}
        """

        motivational_feedback = get_chatgpt_response(resource_prompt)
        print(motivational_feedback)

    # Ask user if they'd like to explore another career path
    while True:
        continue_choice = input("\nWould you like to explore another career path? (yes/no): ").strip().lower()
        if continue_choice == "yes":
            career_goal_tracking(pdf_path, career_titles, smart_goals_dict)
            break
        elif continue_choice == "no":
            quote=get_motivational_quote()
            print(quote)
            print("\nGood luck on your journey!")
            break
        else:
            print("Invalid input. Please enter 'yes' or 'no'.")
pdf_path = 'Charbel-CV.pdf'
career_goal_tracking(pdf_path)


    Career Recommendations for the Candidate:

    Extracted CV Features:
    Certainly! Here's the structured format for the provided CV text:

1. **Contact Information:**
   - **Name:** Charbel C. Abou Younes
   - **Phone Number:** +961 – 70149148
   - **Email:** Charbelabouyounes70@gmail.com
   - **Address:** Beirut, Lebanon

2. **Summary or Objective:**
   - None provided.

3. **Education:**
   - **Degree:** Bachelor’s degree in Electrical and Computer Engineering
   - **Institution:** American University of Beirut (AUB)
   - **Dates:** August 2022 – Present
   - **GPA:** 4.06

4. **Work Experience:**
   - **Position:** Neuroradiology AI Research Assistant
     - **Company:** American University of Beirut – AUB
     - **Dates:** January 2024 – Present
     - **Responsibilities:**
       - Utilized Python to analyze brain images.
       - Fine-tuned AI models related to MS.
       - Supported team decision-making.
   - **Position:** Physics and Math Tutor
     - **Company:** Self-E